## سی بو ڈبلیو ماڈل کی تربیت

یہ نوٹ بک [AI for Beginners Curriculum](http://aka.ms/ai-beginners) کا حصہ ہے۔

اس مثال میں، ہم سی بو ڈبلیو لینگویج ماڈل کی تربیت پر نظر ڈالیں گے تاکہ اپنا Word2Vec ایمبیڈنگ اسپیس حاصل کر سکیں۔ ہم AG News ڈیٹاسیٹ کو متن کے ماخذ کے طور پر استعمال کریں گے۔


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

آئیے پہلے ہمارا ڈیٹاسیٹ لوڈ کریں اور ٹوکنائزر اور وکیبلری کو تعریف کریں۔ ہم `vocab_size` کو 5000 پر سیٹ کریں گے تاکہ حساب کتاب کو تھوڑا محدود کیا جا سکے۔


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## CBoW ماڈل

CBoW ماڈل $2N$ پڑوسی الفاظ کی بنیاد پر ایک لفظ کی پیش گوئی کرنا سیکھتا ہے۔ مثال کے طور پر، جب $N=1$ ہو، تو ہمیں جملے *I like to train networks* سے درج ذیل جوڑے ملیں گے: (like,I)، (I, like)، (to, like)، (like,to)، (train,to)، (to, train)، (networks, train)، (train,networks)۔ یہاں، پہلا لفظ وہ پڑوسی لفظ ہے جو ان پٹ کے طور پر استعمال ہوتا ہے، اور دوسرا لفظ وہ ہے جس کی ہم پیش گوئی کر رہے ہیں۔

اگلے لفظ کی پیش گوئی کے لیے نیٹ ورک بنانے کے لیے، ہمیں پڑوسی لفظ کو ان پٹ کے طور پر فراہم کرنا ہوگا، اور لفظ کا نمبر آؤٹ پٹ کے طور پر حاصل کرنا ہوگا۔ CBoW نیٹ ورک کی ساخت درج ذیل ہے:

* ان پٹ لفظ کو ایمبیڈنگ لیئر سے گزارا جاتا ہے۔ یہی ایمبیڈنگ لیئر ہمارا Word2Vec ایمبیڈنگ ہوگا، اس لیے ہم اسے الگ سے `embedder` ویری ایبل کے طور پر ڈیفائن کریں گے۔ اس مثال میں ہم ایمبیڈنگ سائز = 30 استعمال کریں گے، حالانکہ آپ زیادہ ڈائمینشنز کے ساتھ تجربہ کر سکتے ہیں (حقیقی Word2Vec میں 300 ہوتا ہے)۔
* ایمبیڈنگ ویکٹر کو پھر ایک لینیئر لیئر میں بھیجا جائے گا جو آؤٹ پٹ لفظ کی پیش گوئی کرے گا۔ اس لیے اس میں `vocab_size` نیوران ہوں گے۔

آؤٹ پٹ کے لیے، اگر ہم `CrossEntropyLoss` کو نقصان کے فنکشن کے طور پر استعمال کریں، تو ہمیں متوقع نتائج کے طور پر صرف لفظ کے نمبر فراہم کرنے ہوں گے، بغیر ون-ہاٹ انکوڈنگ کے۔


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## تربیتی ڈیٹا کی تیاری

اب ہم مرکزی فنکشن کو پروگرام کرتے ہیں جو متن سے CBoW لفظی جوڑوں کا حساب لگائے گا۔ یہ فنکشن ہمیں ونڈو سائز متعین کرنے کی اجازت دے گا، اور جوڑوں کا ایک سیٹ واپس کرے گا - ان پٹ اور آؤٹ پٹ الفاظ۔ نوٹ کریں کہ یہ فنکشن الفاظ پر بھی استعمال کیا جا سکتا ہے، اور ویکٹرز/ٹینسرز پر بھی - جو ہمیں متن کو انکوڈ کرنے کی اجازت دے گا، اس سے پہلے کہ اسے `to_cbow` فنکشن میں بھیجا جائے۔


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


چلو تربیتی ڈیٹاسیٹ تیار کرتے ہیں۔ ہم تمام خبروں کا جائزہ لیں گے، `to_cbow` کو کال کریں گے تاکہ لفظی جوڑوں کی فہرست حاصل کی جا سکے، اور ان جوڑوں کو `X` اور `Y` میں شامل کریں گے۔ وقت کی بچت کے لیے، ہم صرف پہلے 10k خبریں مدنظر رکھیں گے - آپ آسانی سے اس حد کو ختم کر سکتے ہیں اگر آپ کے پاس زیادہ وقت ہو انتظار کرنے کے لیے، اور بہتر ایمبیڈنگ حاصل کرنا چاہتے ہوں :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

ہم اس ڈیٹا کو ایک ڈیٹاسیٹ میں تبدیل کریں گے، اور ڈیٹالودر بنائیں گے:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

ہم اس ڈیٹا کو ایک ڈیٹاسیٹ میں تبدیل کریں گے، اور ڈیٹالودر بنائیں گے:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

اب آئیے اصل تربیت کرتے ہیں۔ ہم `SGD` آپٹیمائزر کو کافی زیادہ لرننگ ریٹ کے ساتھ استعمال کریں گے۔ آپ دوسرے آپٹیمائزرز جیسے `Adam` کے ساتھ تجربہ بھی کر سکتے ہیں۔ ہم شروع میں 10 ایپکس کے لیے تربیت کریں گے - اور اگر آپ مزید کم نقصان چاہتے ہیں تو اس سیل کو دوبارہ چلا سکتے ہیں۔


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## ورڈ2ویک آزمانا

ورڈ2ویک استعمال کرنے کے لیے، آئیے اپنے ذخیرہ الفاظ میں موجود تمام الفاظ کے لیے متعلقہ ویکٹر نکالتے ہیں:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

آئیے دیکھتے ہیں، مثال کے طور پر، لفظ **پیرس** کو ایک ویکٹر میں کیسے انکوڈ کیا جاتا ہے:


In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


یہ دلچسپ ہے کہ Word2Vec کا استعمال مترادفات تلاش کرنے کے لیے کیا جائے۔ درج ذیل فنکشن دیے گئے ان پٹ کے لیے `n` قریب ترین الفاظ واپس کرے گا۔ انہیں تلاش کرنے کے لیے، ہم $|w_i - v|$ کا نورم شمار کرتے ہیں، جہاں $v$ ہمارے ان پٹ لفظ کے مطابق ویکٹر ہے، اور $w_i$ ذخیرہ الفاظ میں $i$-ویں لفظ کا انکوڈنگ ہے۔ پھر ہم صف کو ترتیب دیتے ہیں اور `argsort` کا استعمال کرتے ہوئے متعلقہ اشاریے واپس کرتے ہیں، اور فہرست کے پہلے `n` عناصر لیتے ہیں، جو ذخیرہ الفاظ میں قریب ترین الفاظ کے مقامات کو انکوڈ کرتے ہیں۔


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## خلاصہ

چالاک تکنیکوں جیسے کہ CBoW کا استعمال کرتے ہوئے، ہم Word2Vec ماڈل کو تربیت دے سکتے ہیں۔ آپ skip-gram ماڈل کو تربیت دینے کی بھی کوشش کر سکتے ہیں، جو مرکزی لفظ کے دیے جانے پر پڑوسی لفظ کی پیش گوئی کرنے کے لیے تربیت یافتہ ہوتا ہے، اور دیکھ سکتے ہیں کہ یہ کتنی اچھی کارکردگی دکھاتا ہے۔



---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا غیر درستیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ ہم اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے ذمہ دار نہیں ہیں۔
